In [ ]:
import sqlite3
import datetime
def database():
    connection = sqlite3.connect("Bankdetails1.db")
    cursor = connection.cursor()
    cursor.execute("create table if not exists Accounts(AccountNumber int, Name VARCHAR(100),Aadhar VARCHAR(100), DOB VARCHAR(10), balance VARCHAR(10), password VARCHAR(100), type VARCHAR(10))")
    cursor.execute("create table if not exists statements (TransactionId int, Credits int, Debit int, Balance int, transactiondate TEXT, accountno int, toaccount int, remark VARCHAR(1000))")
    a = cursor.execute("select * from  Accounts").fetchall()
    b = cursor.execute("select * from  statements").fetchall()
    connection.commit()
    connection.close()
def create_Accounts():
    connection = sqlite3.connect("Bankdetails1.db")
    cursor = connection.cursor()
    AccountNumber = int(input("Account Number: "))
    Name = input("Name: ")
    Aadhar = input("Aadhar: ")
    DOB = input("DOB (dd-mm-yyyy): ")
    balance = 500  
    password = input("Password: ")
    account_type = input("Account Type: ")
    cursor.execute("insert into Accounts (AccountNumber, Name, Aadhar, DOB, Balance, password, type) VALUES (?, ?, ?, ?, ?, ?, ?)", (AccountNumber, Name, Aadhar, DOB, balance, password, account_type))
    Transaction_Id = int(input("Transaction Id: "))
    credit=int(input("credit"))
    debit=int(input("debit"))
    time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute("insert into statements (TransactionId, Credits, Debit, Balance, transactiondate, accountno, remark) VALUES (?, ?, ?, ?, ?, ?, ?)", (Transaction_Id, credit,debit,balance, time, AccountNumber, "Account created"))
    connection.commit()
    c = cursor.execute("select * from Accounts").fetchall()
    d = cursor.execute("select * from statements").fetchall()
    print(c, d)
    connection.close()
def login():
    connection = sqlite3.connect("Bankdetails1.db")
    cursor = connection.cursor()
    AccountNumber = int(input("Account Number: "))
    password = input("Password: ")
    cursor.execute("select * from Accounts where AccountNumber = ? and password = ?", (AccountNumber, password))
    e = cursor.fetchone()
    print(e)
    try:
        if e!=0:
            print("Success")
        else:
            print("Login failed")
    except Exception as ex:
        print("Failed", ex)
    connection.close()
def process(AccountNumber):
    connection = sqlite3.connect("Bankdetails1.db")
    cursor = connection.cursor()
    while 1:
        n = input("1. Balance 2. Credit 3. Debit 4. Statement 5. Logout: ")
        if n == "1":
            cursor.execute("select Balance from Accounts where AccountNumber = ?", (AccountNumber,))
            balance = cursor.fetchone()
            print(balance)
        elif n == "2":
            credit = int(input("Enter amount to credit: "))
            cursor.execute("select Balance from Accounts where AccountNumber = ?", (AccountNumber,))
            balance = int(cursor.fetchone()[0]) + credit
            cursor.execute("update Accounts set Balance = ? where AccountNumber = ?", (balance, AccountNumber))
            Transaction_Id = cursor.execute("select MAX(TransactionId) from statements").fetchone()[0] + 1
            time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("insert into statements (TransactionId, Credits, Debit, Balance, transactiondate, accountno, remark) VALUES (?, ?5, 0, ?, ?, ?, ?)", 
                           (Transaction_Id, credit, balance, time, AccountNumber, "Credit transaction"))
            connection.commit()
            print("Amount debited, New balance: ",balance)
        elif n == "3":
            debit = int(input("Enter amount to debit: "))
            cursor.execute("select Balance from Accounts where AccountNumber = ?", (AccountNumber,))
            balance = int(cursor.fetchone()[0])
            if balance >= debit:
                balance -= debit
                cursor.execute("update Accounts set Balance = ? where AccountNumber = ?", (balance, AccountNumber))
                Transaction_Id = cursor.execute("select MAX(TransactionId) from statements").fetchone()[0] + 1
                time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                cursor.execute("insert into statements (TransactionId, Credits, Debit, Balance, transactiondate, accountno, remark) VALUES (?, 0, ?, ?, ?, ?, ?)", 
                               (Transaction_Id, debit, balance, time, AccountNumber, "Debit transaction"))
                connection.commit()
                print("Amount debited, New balance: ",balance)
            else:
                print("Insufficient balance")
        elif n == "4":
            cursor.execute("select * from statements where accountno = ?", (AccountNumber,))
            statement = cursor.fetchall()
            print("Transaction history:", statement)
        elif n == "5":
            print("Logout successful")
            break
        else:
            print("Invalid choice")
    connection.close()
database()
f = input("1. Create account 2. Login 3.process ")
if f =='1':
    create_Accounts()
elif f =='2':
    login()
elif f=='3':
    AccountNumber = int(input("Enter your Account Number: "))
    process(AccountNumber)
else:
    print("Invalid choice")
